In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
import pickle
import os
import dataset_utils
import NN

In [9]:
import dataset_utils

In [13]:
def flatten_and_pad(point_cloud, target_length):
    # Flatten the point cloud into a 2D array with X, Y, Z coordinates
    # Ensure the data is padded or truncated to match the target length
    num_points = len(point_cloud)
    padded_point_cloud = np.zeros((target_length, 3))
    padded_point_cloud[:num_points] = point_cloud
    return padded_point_cloud

In [17]:
DATA_DIR = "pointnet_data"
NUM_POINTS = 40
NUM_CLASSES = 2
BATCH_SIZE = 32
train_points, test_points, train_labels, test_labels, CLASS_MAP = dataset_utils.parse_dataset(NUM_POINTS, DATA_DIR)

processing class: sit
processing class: stand


In [18]:
print(train_points)

[[[ 4.678e-01  1.829e-01  1.731e+00  0.000e+00  1.390e+02]
  [-2.656e-01  1.645e+00  7.412e-01  0.000e+00  1.230e+02]
  [-2.983e-01  1.602e+00  9.014e-01  5.010e-01  2.470e+02]
  ...
  [-3.608e-01  1.587e+00  8.525e-01  5.010e-01  3.280e+02]
  [ 4.678e-01  1.816e-01  1.730e+00  0.000e+00  1.370e+02]
  [-1.992e+00  9.424e-01  1.856e+00  0.000e+00  1.840e+02]]

 [[-2.152e+00  2.053e+00  8.721e-01  0.000e+00  2.160e+02]
  [-9.521e-02  1.543e+00  8.486e-01  3.757e-01  2.890e+02]
  [-5.000e-01  1.570e+00  9.385e-01  1.252e-01  1.380e+02]
  ...
  [ 4.678e-01  1.816e-01  1.730e+00  0.000e+00  1.370e+02]
  [-2.152e+00  2.055e+00  8.750e-01  0.000e+00  2.140e+02]
  [-1.992e+00  9.434e-01  1.864e+00  0.000e+00  1.800e+02]]

 [[-2.152e+00  2.051e+00  8.682e-01  0.000e+00  2.120e+02]
  [-2.152e+00  2.053e+00  8.721e-01  0.000e+00  2.160e+02]
  [-1.890e-01  2.381e+00  6.719e-01  3.757e-01  2.220e+02]
  ...
  [-1.992e+00  9.399e-01  1.842e+00  0.000e+00  1.730e+02]
  [-2.152e+00  2.051e+00  8.682e-0

In [19]:
from tensorflow import data as tf_data
import keras
#import PointNET

keras.utils.set_random_seed(seed=42)

In [20]:
train_size = 0.8
dataset = tf_data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf_data.Dataset.from_tensor_slices((test_points, test_labels))
train_dataset_size = int(len(dataset) * train_size)

In [21]:
dataset = dataset.shuffle(len(train_points)).map(NN.augment)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)
train_dataset = dataset.take(train_dataset_size).batch(BATCH_SIZE)
validation_dataset = dataset.skip(train_dataset_size).batch(BATCH_SIZE)

In [22]:
inputs = keras.Input(shape=(NUM_POINTS, 5))

In [11]:
def model_structure_1D(input_shape, output_shape):
    _model_input = Input(shape=input_shape)
    _model = Conv1D(32, kernel_size=1, padding='same', activation='relu')(_model_input)
    _model = MaxPooling1D(pool_size=2)(_model)
    _model = Flatten()(_model)
    _model = Dense(output_shape, activation='softmax')(_model)
    model = Model(inputs=_model_input, outputs=_model)
    return model


In [12]:
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import LinearLocator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Conv3D, MaxPooling3D, Embedding, LSTM, Bidirectional, Reshape, Concatenate

In [13]:
input_shape = (40, 5)  # Example input shape for 3D data
output_shape = 3  # Number of classes or output units
model = model_structure_1D(input_shape, output_shape)

AttributeError: 'tuple' object has no attribute 'rank'

In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()

# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())


# 2. LSTM layer
model.add(layers.LSTM(50, return_sequences=True))
model.add(layers.LSTM(50))

# 3. Dense layers
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(3, activation='linear'))  # Output layer (you can adjust the output size and activation)


In [28]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, None, 64)          1024      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, None, 64)          0         
 g1D)                                                            
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, None, 64)          12352     
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, None, 64)          0         
 g1D)                                                            
                                                                 
 lstm_2 (LSTM)               (None, None, 50)         

In [29]:
# training

history = model.fit(train_points, train_labels, epochs=100, batch_size=32, validation_data=(test_points, test_labels), verbose=4)
test_loss, test_acc = model.evaluate(test_points, test_labels, verbose=2)

pickle.dump(model, open("model.pkl", "wb"))


Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [30]:
print(test_acc)

0.11621193587779999
